In [9]:
import pandas as pd
import numpy as np
import os

In [59]:
## Read in files to get clean base dataframe
directory = "Data"
df_list = []

for root, dirs, files in os.walk(directory):
    if "INFOTABLE.tsv" in files and "OTHERMANAGER2.tsv" in files and "OTHERMANAGER.tsv" in files:
        intotable_path = os.path.join(root, "INFOTABLE.tsv")
        manager1_path = os.path.join(root, "OTHERMANAGER.tsv")
        manager2_path = os.path.join(root, "OTHERMANAGER2.tsv")
        manager1_df = pd.read_csv(manager1_path, sep="\t", usecols=["ACCESSION_NUMBER", "NAME"])
        manager2_df = pd.read_csv(manager2_path, sep="\t", usecols=["ACCESSION_NUMBER", "NAME"])

        manager_df = pd.concat([manager1_df, manager2_df], ignore_index=True)
        manager_df.rename(columns={'NAME':'FUND'}, inplace=True)

        infotable_df = pd.read_csv(intotable_path, sep = "\t", 
                                   usecols=["ACCESSION_NUMBER","NAMEOFISSUER","TITLEOFCLASS",
                                            "CUSIP", "VALUE", "SSHPRNAMT"])
        # Add date
        sub_dir = os.path.basename(root)
        datetime = sub_dir.split("-")[1].split("_")[0]
        infotable_df.insert(0, "Datetime", datetime)
        infotable_df["Datetime"] = pd.to_datetime(infotable_df["Datetime"])
        
        merged_df = pd.merge(infotable_df, manager_df, on="ACCESSION_NUMBER", how="inner") #fund name non-optional
        merged_df.drop(columns=["ACCESSION_NUMBER"], inplace=True)
        df_list.append(merged_df)


df_full = pd.concat(df_list, ignore_index=True)

df_full.set_index(["FUND", "Datetime"], inplace=True)
## 

In [63]:
df_full.loc["METLIFE INC"]

,NAMEOFISSUER,TITLEOFCLASS,CUSIP,VALUE,SSHPRNAMT
Datetime,,,,,
2024-05-31,CREDICORP LTD,COM,G2519Y108,757896,5055
2024-05-31,PAGSEGURO DIGITAL LTD,COM CL A,G68707101,119700,9599
2024-05-31,XP INC,CL A,G98239109,48438,1858
2024-05-31,INTERCORP FINL SVCS INC,SHS,P5626F128,1336645,60895
2024-05-31,COPA HOLDINGS SA,CL A,P31076105,1680442,15807
...,...,...,...,...,...
2024-08-31,ISHARES IBOXX $ HIGH YIELD CORPORA,IBOXX HI Y,464288513,3869728,50165
2024-08-31,ISHARES BROAD USD INVESTMENT GRADE,USD INV GR,464288620,29049899,578568
2024-08-31,SPDR S&P ETF TRUST,TR UNIT,78462F103,111401290,204699


In [72]:
# ADD RANK COLUMN
df_full['RANK'] = df_full.groupby(['FUND', 'Datetime'])['VALUE'].rank(ascending=False, method = 'dense')

In [73]:
df_full.loc["METLIFE INC"].sort_values("RANK")

,NAMEOFISSUER,TITLEOFCLASS,CUSIP,VALUE,SSHPRNAMT,RANK
Datetime,,,,,,
2024-05-31,ISHARES TR,IBOXX INV CP ETF,464287242,921469348,8406800,1.0
2024-08-31,MICROSOFT CORP,COM,594918104,1040306071,2327567,1.0
2024-08-31,APPLE INC,COM,037833100,952286948,4521351,2.0
2024-05-31,ISHARES TR,IBOXX INV CP ETF,464287242,893382191,8473700,2.0
2024-08-31,NVIDIA CORP,COM,67066G104,945075811,7649958,3.0
...,...,...,...,...,...,...
2024-05-31,GUARDANT HEALTH INC,COM,40131M109,0,0,44113.0
2024-05-31,MULLEN AUTOMOTIVE INC,COM,62526P208,0,1,44113.0
2024-05-31,0,0,000000000,0,0,44113.0
